## InMemoryStore

In [1]:
from typing import Any
from langgraph.store.memory import InMemoryStore
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
store = InMemoryStore()
model = ChatGoogleGenerativeAI(model='gemini-2.0-flash')

In [7]:
@tool
def get_user_info(user_id: str, runtime: ToolRuntime) -> Any:
    """Look up user info."""
    store = runtime.store
    user_info = store.get(('Users', ), user_id)
    return str(user_info.value) if user_info else "Unknown user"

In [8]:
@tool
def save_user_info(user_id: str, user_info: dict[str, Any], runtime: ToolRuntime) -> str:
    """Save user info."""
    store = runtime.store
    store.put(("Users",), user_id, user_info)
    return "Successfully saved user info."

In [5]:
agent = create_agent(
	model=model,
	tools=[get_user_info, save_user_info],
    store=store,
	system_prompt="You are a helpful assistant",
)

In [6]:
response1 = agent.invoke({
    "messages": [{"role": "user", "content": "Save the following user: userid: abc123, name: Alice, age: 25, email: alice@langchain.dev"}]
    })

In [11]:
print(response1['messages'][-1].content)

OK. I have saved the user information for user abc123.


In [9]:
response2 = agent.invoke({
    "messages": [{"role": "user", "content": "Get user info for user with id 'abc123'"}]
})

In [12]:
print(response2['messages'][-1].content)

OK. I found user info for user with id 'abc123': {'email': 'alice@langchain.dev', 'age': 25.0, 'name': 'Alice'}


## Checkpointer

In [9]:
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents import create_agent

In [10]:

agent = create_agent(
    model=model,
    tools=[get_user_info],
    checkpointer=InMemorySaver(),  
)

In [ ]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "Hi! My name is Bob."}]},
    {"configurable": {"thread_id": "1"}},
)

In [14]:
print(response['messages'][-1].content)

Hi Bob! How can I help you today?


In [18]:
response = agent.invoke(
	{"messages": [{"role": "user", "content": "what is my name"}]},
	{"configurable": {"thread_id": "1"}},
)

In [19]:
print(response['messages'][-1].content)

I do not have access to your name. However, you just told me your name is Bob.
